In [6]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL
import math
import matplotlib.gridspec as gridspec


import data_transforations as data_tr

from sklearn import tree

# parameters

In [27]:
current_date='2025-08-01'

# loading data

In [28]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"

calls_data = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy_data.csv"))
payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])

## Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

In [29]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)


## Aggregating call, payment, sales and lapse data then merging

In [30]:
pre_inception_calls_data = data_tr.pre_sale_calls(calls_data,policy_data)
post_sale_calls_data = data_tr.post_sale_calls(calls_data,policy_data)
payment_hist_data = data_tr.payments_history(payment_data,current_date=current_date)
lapse_data = data_tr.lapses(lapse_data)
sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')

c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories)
c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories_postsale)
c:\Users\E1005279\OneD

In [31]:
inception_data = data_tr.at_inception_data_merge(pre_inception_calls_data, sales_data,policy_data,lapse_data)

In [6]:

near_ftr_lapse_data = data_tr.post_sale_data_merge(post_sale_calls_data, sales_data,policy_data,lapse_data,payment_hist_data)

## Some cleaning and filtering

In [32]:

inception_data = data_tr.at_inception_data_clean_filter(inception_data)

In [7]:
near_ftr_lapse_data = data_tr.near_ftr_lapse_data_clean_filter(near_ftr_lapse_data)

## Adding economic indicators

In [33]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"
unemployment = pd.DataFrame(pd.read_csv(path+"unemployment rates.csv"))
unemployment= unemployment[['year','ave']]
inflation = pd.DataFrame(pd.read_csv(path+"inflation.csv"))

In [34]:

inception_data['sale_date_year'] = pd.to_datetime(inception_data['sale_date']).dt.year
inception_data = data_tr.adding_eco_ind(inflation, unemployment,inception_data,'sale_date_year')

In [35]:
near_ftr_lapse_data['cover_start_date_year' ] = pd.to_datetime(near_ftr_lapse_data['cover_start_date' ]).dt.year
near_ftr_lapse_data = data_tr.adding_eco_ind(inflation, unemployment,near_ftr_lapse_data,'cover_start_date_year')

NameError: name 'near_ftr_lapse_data' is not defined

## Target creation

In [36]:
inception_data = data_tr.inception_targets(inception_data)


In [10]:
near_ftr_lapse_data = data_tr.near_ftr_targets(near_ftr_lapse_data)

In [11]:
near_ftr_lapse_data.head()

,policy_id,policy_name,age,benefits_count,education,gender,income,smoker_status,sold_socio_economic_class,securitygroup_id,...,# calls contacted,res_code_QA_sum,res_code_claims_sum,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum,inflation,unemployment rate,payment lapse
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,34.300,NaN
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,N,2.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.4,32.600,NaN
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,E,1.0,MWL,...,1.0,0.0,0.0,0.0,3.0,1.0,0.0,4.4,32.600,NaN
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,32.375,NaN
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,N,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,34.300,NaN


In [12]:
near_ftr_lapse_data.to_csv(path+'NearFuture/near_ftr_model_data.csv')

## grouping continous variables

In [37]:
#gropuping function using decision trees
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split


def group_by_decision_tree(model, column_data, column_name):
    """
    Groups a pandas Series into categorical bands based on the leaf nodes of a
    fitted Decision Tree Classifier model.

    This function first traverses the decision tree to determine the value ranges
    for each leaf node. It then applies the model to the input column data to get
    the leaf node index for each data point and uses the collected ranges to
    assign a categorical band.

    Args:
        model (sklearn.tree.DecisionTreeClassifier): A fitted decision tree model.
        column_data (pd.Series): The pandas Series to be grouped.
        column_name (str): The name of the column in the original DataFrame
                           that the decision tree was trained on.

    Returns:
        pd.Series: A new pandas Series containing the categorical bands.
    """
    # Ensure the model is a decision tree
    if not isinstance(model, DecisionTreeClassifier):
        raise TypeError("The provided model must be an instance of DecisionTreeClassifier.")
    
    tree = model.tree_
    
    # Check if the tree is empty or not fitted
    if tree.node_count == 0:
        raise ValueError("The decision tree model does not seem to be fitted.")

    # A helper dictionary to store the min/max range for each leaf node
    leaf_ranges = {}
    
    def traverse_tree(node_index, lower_bound=float('-inf'), upper_bound=float('inf')):
        """
        Recursively traverses the tree to find the value range for each leaf node.
        """
        # Base case: if it's a leaf node
        if tree.children_left[node_index] == tree.children_right[node_index]:
            leaf_ranges[node_index] = (lower_bound, upper_bound)
            return

        threshold = tree.threshold[node_index]
        left_child = tree.children_left[node_index]
        right_child = tree.children_right[node_index]

        # Recurse down the left child (less than or equal to threshold)
        traverse_tree(left_child, lower_bound, min(upper_bound, threshold))

        # Recurse down the right child (greater than threshold)
        traverse_tree(right_child, max(lower_bound, threshold), upper_bound)

    # Start the traversal from the root node (index 0)
    traverse_tree(0)

    # Use the model's apply method to get the leaf node for each data point
    # We need to reshape the data to a 2D array for the model
    leaf_indices = model.apply(column_data.values.reshape(-1, 1))

    # Create the new column of grouped bands
    grouped_bands = []
    for index in leaf_indices:
        lower, upper = leaf_ranges[index]

        # Format the string for the band
        if lower == float('-inf'):
            band = f"< {upper:.2f}"
        elif upper == float('inf'):
            band = f"> {lower:.2f}"
        else:
            band = f"{lower:.2f} - {upper:.2f}"
        
        grouped_bands.append(band)

    return pd.Series(grouped_bands, index=column_data.index, name=f'{column_name}_bands')

In [38]:
cont_var_list=['age','income',  'last_benefit_amount', 'cover_amount_full', 'total_funeral_premium',  'original_premium']
target='lapse_flag'
df=inception_data.copy()
for variable in cont_var_list:
    X = df[variable].values.reshape(-1, 1)
    Y = df[target]
    clf=tree.DecisionTreeClassifier(min_samples_leaf=math.floor(0.15*inception_data.shape[0]))
    clf=clf.fit(X,Y)
    df[variable+'_band'] = group_by_decision_tree(clf, df[variable], df)
inception_data=df

# Exclude policies < 1 year old

In [3]:
from datetime import datetime

In [39]:
inception_data['sale_date']= pd.to_datetime(inception_data['sale_date'])

In [41]:
current_date=datetime.strptime(current_date, "%Y-%m-%d")
year_ago_date = datetime(current_date.year-1,current_date.month, 1)
new_data = inception_data[inception_data['sale_date']>=year_ago_date].copy()
inception_training_data = inception_data[inception_data['sale_date']<year_ago_date].copy()



In [47]:
#inception_data=inception_data.drop(columns=['Unnamed: 0','orginal individual_income', 'policy_id', 'sale_date', 'policy_type',
#       'policy_status','sale_date_year'
#       'current_policy_status', 'cancellation_effective_date',
#       'cancellation_reason', 'lapse_type','cover_start_date', 'end_date', 'pol_duration'
#      ])
inception_training_data.drop(columns=['orginal individual_income', 'policy_id', 'sale_date', 'policy_type',
       'policy_status','sale_date_year',
       'current_policy_status', 'cancellation_effective_date',
       'cancellation_reason', 'lapse_type','cover_start_date', 'end_date', 'pol_duration'
      ],inplace=True)
new_data.drop(columns=['orginal individual_income', 'policy_id', 'sale_date', 'policy_type',
       'policy_status','sale_date_year',
       'current_policy_status', 'cancellation_effective_date',
       'cancellation_reason', 'lapse_type','cover_start_date', 'end_date', 'pol_duration'
      ],inplace=True)

In [48]:
inception_training_data.to_csv(path+'AtIncept-2025/at_incpt_train_model_data.csv')
new_data.to_csv(path+'AtIncept-2025/at_incpt_score_model_data.csv')

In [49]:
new_data

,policy_name,age,education,gender,income,smoker_status,sold_socio_economic_class,occupation_class,occupation,lead_provider_name,...,6month_lapse,1yr_lapse,ann_lapse,2yr_lapse,age_band,income_band,last_benefit_amount_band,cover_amount_full_band,total_funeral_premium_band,original_premium_band
1,ML24-0254853,63,Matric + 3 year Diploma,Female,45000.0,E,1.0,A,Personal Assistant,SPL-Freemium,...,0,0,NaN,0,> 50.50,> 37462.50,< 110120.46,< 100030.60,< 8.27,280.76 - 385.28
7,ML24-0293644,51,Matric,Male,55000.0,N,1.0,B,Mining - Underground - Mine Manager (< 20 hour...,MWL Phone Leads,...,0,0,NaN,0,> 50.50,> 37462.50,498472.00 - 1003478.84,200015.62 - 477116.67,> 163.69,> 596.20
11,ML24-0276881,30,Matric,Male,35000.0,S,2.0,A,Sales Representative (< 10% travel),Sold Freemium,...,0,0,NaN,0,< 32.50,26846.00 - 37462.50,498472.00 - 1003478.84,477116.67 - 1005833.34,8.27 - 75.47,385.28 - 596.20
15,ML24-0280064,58,No Matric,Male,300000.0,N,1.0,C,Director (< 10% travel and 21% - 50% manual work),SPL-Freemium,...,0,0,NaN,0,> 50.50,> 37462.50,498472.00 - 1003478.84,200015.62 - 477116.67,8.27 - 75.47,> 596.20
17,ML24-0278890,39,Matric + certificate,Female,28000.0,N,2.0,A,Secretary,Website,...,0,0,NaN,0,38.50 - 43.50,26846.00 - 37462.50,498472.00 - 1003478.84,477116.67 - 1005833.34,75.47 - 163.69,385.28 - 596.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22319,ML24-0282083,56,Matric + 4 year Diploma,Male,26000.0,E,2.0,D,Police - Warrant officer (Previously called In...,Sold Freemium,...,0,0,NaN,0,> 50.50,16194.50 - 26846.00,< 110120.46,< 100030.60,< 8.27,385.28 - 596.20
22324,ML24-0296656,38,Matric,Male,22000.0,N,2.0,E,Rigger (Heights up to 10m),Quoted Freemium,...,0,0,NaN,0,32.50 - 38.50,16194.50 - 26846.00,498472.00 - 1003478.84,200015.62 - 477116.67,> 163.69,385.28 - 596.20
22326,ML25-0340157,34,4 year Degree,Male,30000.0,S,1.0,B,Teacher,Facebook,...,1,1,NaN,1,32.50 - 38.50,26846.00 - 37462.50,498472.00 - 1003478.84,477116.67 - 1005833.34,< 8.27,280.76 - 385.28
22328,ML25-0319841,49,Matric + 4 year Diploma,Female,42000.0,N,1.0,C,Roadworthy Inspector,Sold Freemium,...,0,0,NaN,0,43.50 - 50.50,> 37462.50,> 1666875.00,> 1005833.34,< 8.27,> 596.20
